Structure:
* find tile corresponding to geojson
* download B8A, B11 tiles
* calculate NDVI
* threshold vegetation
* classify

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub import AwsProductRequest
from geojson import Polygon
from datetime import date
import json
import folium
import geopandas as gpd
import pandas as pd
import requests
import fiona

# Get GeoJSONs

In [9]:
data = requests.get('https://raw.githubusercontent.com/drakh/slovakia-gps-data/master/GeoJSON/epsg_4326/slovakia_esri_epsg_4326.geojson')
b = bytes(data.content)
with fiona.BytesCollection(b) as f:
    crs = f.crs
    svk_gdf = gpd.GeoDataFrame.from_features(f, crs=crs)

geojson = svk_gdf.geometry[0]

In [65]:
footprint = geojson.convex_hull.to_wkt()  # geojson.to_wkt() TOO LONG

In [56]:
dates_L1C = (date(2007, 1, 1), date(2017, 5, 2))  # Sentinel 2 Level 1C
dates_L2A = (date(2017, 5, 2), 'NOW')  # Sentinel 2 Level 2A

weeks_L2A = pd.date_range(start=date(2017, 5, 2), end='NOW', freq='1W-MON').to_pydatetime()
weeks_L2A = [x.date() for x in weeks_L2A]
weeks_L2A = list(zip(weeks_L2A[:-1], weeks_L2A[1:]))

weeks_L2A[0]

(datetime.date(2017, 5, 8), datetime.date(2017, 5, 15))

# Get corresponding product ids

In [72]:
len(weeks_L2A)

50

In [71]:
product_ids = get_sentinel_products(dates_L2A, producttype=L2A, area_relation=INTERSECTS)
len(product_ids)

81

In [ ]:
product_id = product_ids[0]  # June

# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
geo_df = api.to_geodataframe(products).loc[[product_id]][['geometry']]

def add_choropleth(mapobj, gdf, fill_color='Blue', fill_opacity=0.6,
                   line_opacity=0.2, num_classes=5):
    # Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    # Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON,
    # the data as the GeoDataFrame, the columns as the user-specified id field and and value field.
    # key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(
        geo_data=gdf_wgs84.to_json(),
        fill_color=fill_color,
        fill_opacity=fill_opacity,
        line_opacity=line_opacity,  
    )
    return mapobj

slovakia_map = folium.Map(np.mean(coordinates[0], 0).tolist()[::-1], zoom_start=8)
slovakia_map = add_choropleth(slovakia_map, geo_df)
slovakia_map

# Download tiles

In [ ]:
bands = ['B8A', 'B11']
metafiles = ['tileInfo', 'preview', 'qi/MSK_CLOUDS_B00']
data_folder = '../data/AwsData'

request = AwsTileRequest(tile=tile_name, time=time, aws_index=aws_index, 
                         bands=bands, metafiles=metafiles, data_folder=data_folder)
request.save_data()

data_list = request.get_data()
b8a, b11, tile_info, preview, cloud_mask = data_list

In [ ]:
plt.imshow(preview)

In [41]:
product_id = 'S2A_MSIL1C_20171010T003621_N0205_R002_T01WCV_20171010T003615'
# product_id = 'S2A_OPER_PRD_MSIL1C_PDMC_20160121T043931_R069_V20160103T171947_20160103T171947'
data_folder = '../data/AwsData'
bands = ['B8A', 'B11']
bands = None

product_request = AwsProductRequest(product_id=product_id, bands=bands, data_folder=data_folder, safe_format=True)
# data_list = product_request.get_data(save_data=True)
product_request.save_data()

# product_request = AwsProductRequest(product_id=product_id, bands=['B8A', 'B11'], data_folder=data_folder)
# data_list = product_request.get_data(save_data=True)

# Post-process with Sen2Cor if product-type is S2MSI1C

In [25]:
import os

In [32]:
data_path = os.path.join(data_folder, product_id + '.SAFE')
data_path

'../data/AwsData/S2A_OPER_PRD_MSIL1C_PDMC_20160121T043931_R069_V20160103T171947_20160103T171947.SAFE'

In [33]:
ll -h $data_path

total 732K
drwxr-xr-x  2 adrian 4,0K 29 apr 12:32 AUX_DATA/
drwxr-xr-x  3 adrian 4,0K 29 apr 12:32 DATASTRIP/
drwxr-xr-x 14 adrian 4,0K 29 apr 12:32 GRANULE/
drwxr-xr-x  2 adrian 4,0K 29 apr 12:32 HTML/
-rw-r--r--  1 adrian  20K 29 apr 12:32 INSPIRE.xml
-rw-r--r--  1 adrian 635K 29 apr 12:32 manifest.safe
drwxr-xr-x  2 adrian 4,0K 29 apr 12:32 rep_info/
-rw-r--r--  1 adrian 7,1K 29 apr 12:32 S2A_OPER_BWI_MSIL1C_PDMC_20160121T043931_R069_V20160103T171947_20160103T171947.png
-rw-r--r--  1 adrian  48K 29 apr 12:32 S2A_OPER_MTD_SAFL1C_PDMC_20160121T043931_R069_V20160103T171947_20160103T171947.xml


In [34]:
sencor_cmd = '/home/adrian/PycharmProjects/sentinel/Sen2Cor-02.05.05-Linux64/bin/L2A_Process'

In [36]:
!$sencor_cmd $data_path

Operation mode PDGS is not supported for Product version 13.1.
Operation mode will be reset to TOOLBOX.

Sentinel-2 Level 2A Processor (Sen2Cor). Version: 2.5.5, created: 2018.03.19, supporting Level-1C product version <= 14.5 started ...
Product version: 13.1. Operation mode: TOOLBOX. Processing baseline: 02.01.
/home/adrian/PycharmProjects/sentinel/Sen2Cor-02.05.05-Linux64/bin/L2A_Process: line 5: 16330 Segmentation fault      (core dumped) /home/adrian/PycharmProjects/sentinel/Sen2Cor-02.05.05-Linux64/bin/python2.7 -s /home/adrian/PycharmProjects/sentinel/Sen2Cor-02.05.05-Linux64/lib/python2.7/site-packages/sen2cor/L2A_Process.py "$@"


# trva 18 minut na jeden tile!

# Classify